# Zigmoid

In this example, we generate a 2x2 matrix of values selected to demonstrate the different piecewise separated areas of interest for the zigmoid function. Then we will reveal them so we can check correctness of the protocol.

Next we will call the zigmoid function on the secret_share which will calculate the result of applying the function to the matrix on an elementwise basis and return the shared result. This we reveal and decode and print to check against what we would expect given the original secrets.

As you can see from inspection the result of the protocol satifies the desired functionality of what one would expect from the so-called zigmoid which is an approximation of sigmoid which is more angular and much easier to compute in the SMC context.

The zigmoid function is defined thusly:

$$
zigmoid(x) = \left\{
    \begin{array}\\
        0 & if\ x<-0.5 \\
        x+0.5 & if\ -0.5\leq x \leq 0.5 \\
        1 & if x>0.5
    \end{array}
    \right.
$$

It is analagous to the "cut" function defined in the literature and the same as similar approximations used elsewhere such as the in work given in the SecureML paper.

In [1]:
import logging

import numpy

import cicada.additive
import cicada.communicator

logging.basicConfig(level=logging.INFO)

@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)
    generator = numpy.random.default_rng()
    secret_share = protocol.share(secret=protocol.encoder.encode(numpy.array([[-5,-.25],[.25,5]])), src=0, shape=(2,2))
    secret = protocol.encoder.decode(protocol.reveal(secret_share))
    log.info(f"Player {communicator.rank} secret: \n{secret}")
    zigmoid_share = protocol.zigmoid(secret_share)
    zig = protocol.encoder.decode(protocol.reveal(zigmoid_share))
    log.info(f"Player {communicator.rank} zigmoid(secret): \n{zig}")

main();

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:58465 from tcp://127.0.0.1:58465.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:58465 from tcp://127.0.0.1:58466.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:58465 from tcp://127.0.0.1:58467.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:root:Player 0 secret: 
[[-5.   -0.25]
 [ 0.25  5.  ]]
INFO:root:Player 1 secret: 
[[-5.   -0.25]
 [ 0.25  5.  ]]
INFO:root:Player 2 secret: 
[[-5.   -0.25]
 [ 0.25  5.  ]]
INFO:root:Player 0 zigmoid(secret): 
[[0.   0.25]
 [0.75 1.  ]]
INFO:root:Player 1 zigmoid(secret): 
[[0.   0.25]
 [0.75 1.  ]]
INFO:root:Player 2 zigmoid(secret): 
[[0.   0.25]
 [0.75 1.  ]]
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.nng:Comm '